In [10]:
import pandas as pd
import numpy as np
import sqlite3
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [2]:
timeline = pd.read_csv('./crypto_reg_timeline.csv')
timeline['Date'] = pd.to_datetime(timeline['Date'], format='%Y%m%d')

In [3]:
timeline.tail()

,Date,Country/Region,Topic
19,2022-11-02,SP,First DeFi trades in wholesale markets of gove...
20,2022-12-16,Global,BCBS final standard on prudential treatment fo...
21,2023-01-27,US,FED wants to limit activitiy of SMBs from hold...
22,2023-03-12,US,SVB and Signature Bank crash
23,2023-06-29,EU,EU Parliament issued MiCAr legislation on cryp...


In [4]:
tweets = pd.read_csv('Bitcoin_tweets_dataset_2.csv', lineterminator='\n')
tweets.head()

,user_name,user_location,user_description,user_created,user_followers,user_friends,user_favourites,user_verified,date,text,hashtags,source,is_retweet
0,ChefSam,Sunshine State,Culinarian | Hot Sauce Artisan | Kombucha Brew...,2011-03-23 03:50:13,4680,2643,6232,False,2023-03-01 23:59:59,Which #bitcoin books should I think about read...,['bitcoin'],Twitter for iPhone,False
1,Roy⚡️,NaN,Truth-seeking pleb 📚 • Science 🧪 • Nature 🌱☀️ ...,2022-01-30 17:41:41,770,1145,9166,False,2023-03-01 23:59:47,"@ThankGodForBTC I appreciate the message, but ...",['Bitcoin'],Twitter for iPhone,False
2,Ethereum Yoda,NaN,UP or DOWN...\n.\n.\n.\n.\nPrice matters NOT.,2022-07-24 04:50:18,576,1,0,False,2023-03-01 23:59:42,#Ethereum price update: \n\n#ETH $1664.02 USD\...,"['Ethereum', 'ETH', 'Bitcoin', 'BTC', 'altcoin...",Twitter Web App,False
3,Viction,"Paris, France",https://t.co/8M3rgdjwEe\n\n#bitcoin #blockchai...,2010-03-26 10:15:26,236,1829,2195,False,2023-03-01 23:59:36,CoinDashboard v3.0 is here\nAvailable on ios a...,['Bitcoin'],Twitter for Android,False
4,Rosie,London,"The flower language of jasmine is loyalty, res...",2013-02-16 09:57:56,12731,46,134,False,2023-03-01 23:59:32,#Bitcoin Short Term Fractal (4H)💥\n\nIn lower ...,"['Bitcoin', 'BTC']",Twitter Web App,False


In [5]:
# tweets['date'] = tweets['date'].str[:10]
tweets['date'] = pd.to_datetime(tweets['date'], format='%Y-%m-%d %H:%M:%S')
tweets.sort_values(by='date', ascending=False, inplace=True)
# tweets.rename(columns={'date': 'Date'}, inplace=True)

# merged_df = pd.merge(timeline, tweets, on='Date', how='inner')

In [6]:
#sentiment analysis for postive or negative
nltk.download('vader_lexicon')

def get_sentiment(text):
    analyzer = SentimentIntensityAnalyzer()
    sentiment = analyzer.polarity_scores(text)
    return sentiment['compound']
    
tweets['sentiment_score'] = tweets['text'].apply(get_sentiment)

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/MicheleOrlandi/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
tweets_clean = tweets.groupby(pd.Grouper(key='date', freq='15min')).agg({'sentiment_score': 'mean'}).reset_index()
tweets_clean['date'] = pd.to_datetime(tweets_clean['date'], format='%Y-%m-%d %H:%M:%S')

In [8]:
tweets_clean.head()

,date,sentiment_score
0,2023-02-25 20:45:00,0.172491
1,2023-02-25 21:00:00,0.140640
2,2023-02-25 21:15:00,0.093906
3,2023-02-25 21:30:00,0.133207
4,2023-02-25 21:45:00,0.145924


In [9]:
engine = sqlite3.connect('./btc-data.db')
btc = pd.read_sql('SELECT * FROM BTCUSD', engine)

In [10]:
btc.head()

,date,open,high,low,close,volume,transactions,log,logRet
0,2020-01-01 00:15:00,7182.78,7198.400000,7151.60,7157.00,225.209649,790,8.875846,-0.000172
1,2020-01-01 00:30:00,7194.20,7200.004497,7155.00,7155.72,84.017938,617,8.875667,-0.000179
2,2020-01-01 00:45:00,7158.20,7201.900000,7149.25,7155.70,187.857548,809,8.875665,-0.000003
3,2020-01-01 01:00:00,7150.65,7206.100000,7147.18,7167.34,336.479456,729,8.877290,0.001625
4,2020-01-01 01:15:00,7192.40,7226.100000,7163.92,7186.60,410.926622,968,8.879973,0.002684


In [11]:
btc_clean = btc.loc[:, ['date', 'logRet']].copy()
btc_clean['date'] = pd.to_datetime(btc_clean['date'], format='%Y-%m-%d %H:%M:%S')
merged = btc_clean.merge(tweets_clean, on='date', how='inner')

In [26]:
merged.dropna(axis=0, inplace=True)

In [192]:
merged.to_csv('./sentiment.csv', index=False)

In [12]:
merged

,date,logRet,sentiment_score
0,2023-02-25 20:45:00,0.001453,0.172491
1,2023-02-25 21:00:00,-0.000785,0.140640
2,2023-02-25 21:15:00,0.001055,0.093906
3,2023-02-25 21:30:00,-0.001289,0.133207
4,2023-02-25 21:45:00,0.001479,0.145924
...,...,...,...
677,2023-03-05 22:45:00,0.000130,0.124058
678,2023-03-05 23:00:00,-0.001184,0.142931
679,2023-03-05 23:15:00,-0.000276,0.142951
680,2023-03-05 23:30:00,0.000755,0.156057
